In [2]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from random import randint
import time
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import requests

In [3]:
#Defines list of icon ID values that don't have useful info
pointlessicons = ['badge-with-icon', 'badge-secondary']

#Creates Pandas dataframe with the metadata
hannibal_df = pd.DataFrame(columns=["Title", "Storylink", "AuthName", "AuthID", "Shiptype", "Rating",  "Likes", "Fandom", "Charships", "Length", "Postdate", "Tags", "Reviews", "Followers", "Collections", "Content"])

#Launches the Undetected Selenium driver
driver = uc.Chrome()

In [4]:
#Base URL for a fandom, up to `?p=` which defines which page
ScraperStem = "https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p="
BaseURL = "https://ficbook.net"
headers = {'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36'}

In [ ]:
#Set the range here, between 1 and the highest-number page for the fandom
for i in range(1,239):
    #Define the full URL as the base URL + the page number
    ScraperURL = ScraperStem + str(i)
    #Print the full URL
    print(ScraperURL)
    #Load the full URL
    driver.get(ScraperURL);
    #Wait 6 seconds
    time.sleep(6)
    #Find all the spoiler / hidden tags
    spoilers = driver.find_elements(by=By.CLASS_NAME, value='show-hidden-tags-btn')
    #For every spoiler / hidden tag, click the tag to show its value
    for x in range(0,len(spoilers)):
        if spoilers[x].is_displayed():
            spoilers[x].click()
            sleep(randint(1,3))
    #Get the page source
    pageSource = driver.page_source
    #Parse the page source with Beautiful Soup
    soup = BeautifulSoup(pageSource)
    #Find the container for the fics
    fics = soup.find_all("div", {'class': 'js-toggle-description'})
    #For each fic
    for fic in fics:
        #Find the title container
        title = fic.find('h3', {'class': 'fanfic-inline-title'})
        #Find the story link
        storylink = title.a['href']
        #Extracting from inside the link
        story_page = BaseURL + storylink
        #Load the full URL
        driver.get(story_page);
        #driver.find_element(By.ID, "adultCoverWarningHide").click()
        #Get the page source
        #driver.page_source
        fanficsource = driver.page_source
        fanficsoup = BeautifulSoup(fanficsource)
        #Parse the page source with Beautiful Soup
        for element in fanficsoup:
            text_folder = ""
            reviews = element.find('a', {'class': 'btn btn-primary btn-with-description'})
            reviews = reviews.find('span', {'class': 'main-info'}).text.strip()
            followers = element.find('button',{'class':'btn btn-with-description js-follow-fanfic jsVueComponent btn-primary'})
            followers = followers.find('span', {'class': 'main-info'}).text
            collections = element.find_all('span', {'class': 'btn btn-primary btn-with-description'})[1]
            collections = collections.find('span', {'class': 'main-info'}).text
            if element.find('ul', {'class': 'list-unstyled list-of-fanfic-parts clearfix'}):
                chapters_list = element.find_all('li', {'class': 'part'})
                for chapter in chapters_list:
                    link = chapter.a['href']
                    chapter_page = BaseURL + link
                    driver.get(chapter_page);
                    chaptersource = driver.page_source
                    chaptersoup = BeautifulSoup(chaptersource)
                    for part in chaptersoup:
                        text = part.find('div', {'id': 'content'}).text
                        text_folder += text
            else: 
                text = element.find('div', {'id': 'content'}).text
                text_folder += text
        #Find the text of the title
        title = title.text
        #Find the container with the ship type
        shiptype = fic.find('div', {'class': 'direction'})
        #Find the span with the ship type badge 
        shiptype = shiptype.find('span', {'class': 'badge-text'})
        #Get the text from the ship type
        shiptype = shiptype.text
        #Get all the containers with icons
        icons = fic.find_all("span", {'class': 'badge-with-icon'})
        #For each icon
        for icon in icons:
            #Get the class of the icon as the iconvalue
            iconvalues = icon["class"]
            #For each iconvalue
            for iconvalue in iconvalues:
                #If the iconvalue isn't in the pointlessicons list
                if iconvalue not in pointlessicons:
                    #If the iconvalue includes 'badge-status'
                    if 'badge-status' in iconvalue:
                        #The iconvalue is the text status
                        textstatus = iconvalue
        #Find a span tag with an icon
        rating = fic.find('div', class_=re.compile(r'badge-rating-'))
        if rating not in pointlessicons:
            #Get the badge text as the rating
            rating = rating.find('span', {'class': 'badge-text'}).text
        #Find a span with the class badge-likes
        likes = fic.find('div', {"class": 'badge-like'})
        if likes is not None:
            #If it's not empty, that's the number of likes
            likenumber = likes.find('span', {"class": 'badge-text'}).text
        else:
            #Otherwise assign it to empty
            likenumber = ''
        #Find a span with the class badge-translate
        translationicon = fic.find('div', {'class': 'badge-translate'})
        #If it's not empty, then it's translated
        if translationicon is not None:
            translation = 'translated'
        else:
            #Otherwise set translation to empty
            translation = ''
        #Find a span with the class badge-reward
        award = fic.find('div', {"class": 'badge-reward'})
        #If it's not empty, that's the award
        if award is not None:
            awardnumber = award.find('span', {"class": 'badge-text'}).text
        else:
            #Otherwise set award to empty
            award = ''
        #Find a span with the class author
        author = fic.find('span', {'class': 'author'})
        #Author profile link is the link here
        authlink = author.a['href']
        #Author name is the text on this link
        authname = author.text
        #Find the table with metadata
        tables = fic.find_all('dd')
        #The fandom is the second value in the table
        fandom = tables[1].text
        #If there's 5 things in the table:
        if len(tables) == 4:
            #Set character ships to empty
            charships = ''
            #Length is the third value in the table
            length = tables[2].text
            #Post date is the 4th value in teh table
            postdate = tables[3].text
        #If there's 6 things in the table
        if len(tables) == 5:
            #Character ships are the third thing
            charships = tables[2].text
            #Length is the 4th thing
            length = tables[3].text
            #Post date is the 5th thing
            postdate = tables[4].text
        #Find a div with the class tags
        tagbox = fic.find("div", {"class": 'tags'})
        #If there are tags
        if tagbox is not None:
            #Find all links
            taglist = tagbox.find_all('a')
            #Make an empty list for tags
            tags = []
            #For each tag in the tag list
            for tag in taglist:
                #The tag is the text of the link in the tag box
                tag = tag.text
                #Add that tag to the list of tags
                tags.append(tag)
                #Combine all the things in the tag list, separated by pipes
                alltags = '|'.join(tags)
        #Create a new item with the metadata that's been scraped
        newitem2 = {"Title": title, "Storylink": storylink, "AuthName": authname, "AuthID": authlink, "Shiptype": shiptype, "Rating": rating,"Likes": likenumber,"Fandom": fandom, "Charships": charships, "Award": award, "Translation": translation, "Length": length, "Postdate": postdate, "Tags": alltags, "Reviews": reviews, "Followers": followers, "Collections": collections, "Content": text_folder}
        #Add the item to the Pandas dataframe
        hannibal_df = hannibal_df.append(newitem2, ignore_index=True)
        
    #Wait 3-10 seconds before loading the new page
    sleep(randint(3,10))

https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=1
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=2
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=3
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=4
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=5
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=6
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=7
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=8
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=9
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=10
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=11
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=12
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=13
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=14
https://ficbook.net/fanfiction/movies_and_tv_series/gannibal_?p=15
http

In [5]:
#Display the results
hannibal_df = hannibal_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=False)
hannibal_df

,Title,Storylink,AuthName,AuthID,Shiptype,Rating,Likes,Fandom,Charships,Length,Postdate,Tags,Reviews,Followers,Collections,Content


In [ ]:
hannibal_df_try.to_csv("file5.csv", sep=',')

In [ ]:
hannibal_df_try